In [1]:
import os
from PIL import Image
import piexif

In [2]:
input_dir = os.getcwd()  # текущая директория
output_dir = os.path.join(input_dir, 'compressed_photos')  # сюда будут сохранены сжатые фото
os.makedirs(output_dir, exist_ok=True)

# Допустимые форматы
extensions = ('.jpg', '.jpeg', '.png', '.heic', '.heif')

# Коэффициент сжатия (чем меньше — тем сильнее сжимает)
QUALITY = 15  # от 1 до 95, оптимально 60–80

# Проверка поддержки HEIC
try:
    import pillow_heif
    pillow_heif.register_heif_opener()
except ImportError:
    print("⚠️ Для поддержки HEIC установите библиотеку: pip install pillow-heif")

### Рекурсивный обход всех подпапок и сжатие изображений

In [3]:
for root, dirs, files in os.walk(input_dir):
    for file in files:
        if file.lower().endswith(extensions):
            src_path = os.path.join(root, file)

            # формируем путь назначения (внутри compressed_photos, с сохранением структуры подпапок)
            rel_path = os.path.relpath(src_path, input_dir)
            dst_path = os.path.join(output_dir, rel_path)
            img_name = dst_path.split('\\')[-1]

            # создаём промежуточные папки, если их нет
            dst_dir = os.path.dirname(dst_path)
            os.makedirs(dst_dir, exist_ok=True)

            try:
                # открываем и конвертируем
                img = Image.open(src_path)
                
                # пробуем извлечь EXIF (если есть)
                exif = img.info.get('exif')
                
                img = img.convert('RGB')

                # сохраняем как JPEG
                dst_jpg = os.path.splitext(dst_path)[0] + '_comp.jpg'
                if exif:
                    img.save(dst_jpg, 'JPEG', quality=QUALITY, optimize=True, exif=exif)
                else:
                    img.save(dst_jpg, 'JPEG', quality=QUALITY, optimize=True)
                    print('У фото ', img_name, ' не сохранены метаданные')

                print(f"✅ Сжато: {src_path} → {dst_jpg}")

            except Exception as e:
                print(f"❌ Ошибка при обработке {src_path}: {e}")


✅ Сжато: d:\RUSLAN\WORK\DOM_RF\Veliky_Ustyug\okn\webmap\git-repo\okn-map-VU\photos\IMG_6079.heic → d:\RUSLAN\WORK\DOM_RF\Veliky_Ustyug\okn\webmap\git-repo\okn-map-VU\compressed_photos\photos\IMG_6079_comp.jpg
✅ Сжато: d:\RUSLAN\WORK\DOM_RF\Veliky_Ustyug\okn\webmap\git-repo\okn-map-VU\photos\IMG_6080.heic → d:\RUSLAN\WORK\DOM_RF\Veliky_Ustyug\okn\webmap\git-repo\okn-map-VU\compressed_photos\photos\IMG_6080_comp.jpg
✅ Сжато: d:\RUSLAN\WORK\DOM_RF\Veliky_Ustyug\okn\webmap\git-repo\okn-map-VU\photos\IMG_6136.heic → d:\RUSLAN\WORK\DOM_RF\Veliky_Ustyug\okn\webmap\git-repo\okn-map-VU\compressed_photos\photos\IMG_6136_comp.jpg
✅ Сжато: d:\RUSLAN\WORK\DOM_RF\Veliky_Ustyug\okn\webmap\git-repo\okn-map-VU\photos\IMG_6137.heic → d:\RUSLAN\WORK\DOM_RF\Veliky_Ustyug\okn\webmap\git-repo\okn-map-VU\compressed_photos\photos\IMG_6137_comp.jpg
✅ Сжато: d:\RUSLAN\WORK\DOM_RF\Veliky_Ustyug\okn\webmap\git-repo\okn-map-VU\photos\IMG_6138.heic → d:\RUSLAN\WORK\DOM_RF\Veliky_Ustyug\okn\webmap\git-repo\okn-map-V

### Создание geojson с точками фото

In [60]:
import os
import json
from PIL import Image, ExifTags
from GPSPhoto import gpsphoto

# === Настройки ===
os.chdir(r"D:\RUSLAN\WORK\DOM_RF\Veliky_Ustyug\okn\webmap\git-repo\okn-map-VU\compressed_photos")
input_dir = os.getcwd()  # Рабочая директория
print(input_dir)
output_geojson = os.path.join(input_dir, "photos.geojson")

extensions = ('.jpg')

def get_gps_coords(img_path):
    """Извлекает GPS координаты из EXIF в десятичном формате."""
    data = gpsphoto.getGPSData(img_path)
    lat = data['Latitude']
    lon = data['Longitude']
    if not lat:
        return None

    # def to_decimal(val):
    #     # Если уже float, просто возвращаем
    #     if isinstance(val[0], float):
    #         d, m, s = val
    #         return d + m / 60 + s / 3600
    #     # Иначе предположим, что кортеж ((num,den),...)
    #     d = val[0][0] / val[0][1]
    #     m = val[1][0] / val[1][1]
    #     s = val[2][0] / val[2][1]
    #     return d + m / 60 + s / 3600

    # try:
    #     lat = to_decimal(gps_info["GPSLatitude"])
    #     lon = to_decimal(gps_info["GPSLongitude"])

    #     if gps_info.get("GPSLatitudeRef") == "S":
    #         lat = -lat
    #     if gps_info.get("GPSLongitudeRef") == "W":
    #         lon = -lon

    return lat, lon
    # except KeyError:
        # return None


# --- Основная логика ---
features = []

for root, _, files in os.walk(input_dir):
    for file in files:
        if file.lower().endswith(extensions):
            file_path = os.path.join(root, file)
            rel_path = os.path.relpath(file_path, input_dir).replace("\\", "/")

            try:
                coords = get_gps_coords(file_path)

                if coords:
                    lat, lon = coords
                    features.append({
                        "type": "Feature",
                        "geometry": {"type": "Point", "coordinates": [lon, lat]},
                        "properties": {"photo": rel_path}
                    })
                    print(f"✅ {rel_path} — {lat:.6f}, {lon:.6f}")
                else:
                    print(f"⚠️ Нет геотега: {rel_path}")

            except Exception as e:
                print(f"❌ Ошибка {file_path}: {e}")

# --- Сохранение результата ---
geojson = {"type": "FeatureCollection", "features": features}

with open(output_geojson, "w", encoding="utf-8") as f:
    json.dump(geojson, f, ensure_ascii=False, indent=2)

print(f"\n📍 Найдено {len(features)} фото с геотегами")
print(f"💾 GeoJSON сохранён в {output_geojson}")


D:\RUSLAN\WORK\DOM_RF\Veliky_Ustyug\okn\webmap\git-repo\okn-map-VU\compressed_photos
✅ photos/IMG_6079_comp.jpg — 60.770103, 46.322422
✅ photos/IMG_6080_comp.jpg — 60.770061, 46.322542
✅ photos/IMG_6136_comp.jpg — 60.755950, 46.317039
✅ photos/IMG_6137_comp.jpg — 60.755325, 46.317781
✅ photos/IMG_6138_comp.jpg — 60.755256, 46.317950
❌ Ошибка D:\RUSLAN\WORK\DOM_RF\Veliky_Ustyug\okn\webmap\git-repo\okn-map-VU\compressed_photos\photos\IMG_6139_comp.jpg: 'Latitude'
✅ photos/IMG_6140_comp.jpg — 60.755467, 46.315922
✅ photos/IMG_6142_comp.jpg — 60.755194, 46.315036
✅ photos/IMG_6143_comp.jpg — 60.755194, 46.315006
✅ photos/IMG_6144_comp.jpg — 60.755006, 46.314372
✅ photos/IMG_6145_comp.jpg — 60.755000, 46.314369
❌ Ошибка D:\RUSLAN\WORK\DOM_RF\Veliky_Ustyug\okn\webmap\git-repo\okn-map-VU\compressed_photos\photos\IMG_6146_comp.jpg: 'Latitude'
✅ photos/IMG_6147_comp.jpg — 60.753911, 46.314653
✅ photos/IMG_6148_comp.jpg — 60.753917, 46.314603
✅ photos/IMG_6149_comp.jpg — 60.753656, 46.314803
✅ p